In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [10]:
import pandas as pd
from pandas import DataFrame, Series
from datetime import datetime, timedelta

Interesting thing from the start, he lays out three ways of referring to time that he's going to discuss in the chapter (timestamps, fixed periods - special case of intervals, and intervals), and then also says there's 'experiment or elapsed time', which is where you have a timestamp that's a measure of time relative to a particular start time... like the number of hours from when a machine has first boot. He focuses on the first three, but says you can 'apply many of the techniques' in the chapter where you have the index be an integer for floating point number that shows the elapsed time from the start of the experiment. Ok.

#Date and time data types and tools

In [3]:
now = datetime.now()
now

datetime.datetime(2015, 8, 30, 14, 24, 49, 346209)

In [4]:
now.year, now.month, now.day

(2015, 8, 30)

In [6]:
delta = datetime(2011,1,7) - datetime(2008,6,24,8,15)
delta

datetime.timedelta(926, 56700)

In [8]:
delta.days, delta.seconds

(926, 56700)

In [12]:
start = datetime(2011,1,7)
start + timedelta(12) # 12 days

datetime.datetime(2011, 1, 19, 0, 0)

In [13]:
start - 2 * timedelta(12)

datetime.datetime(2010, 12, 14, 0, 0)

##Converting between string and datetime

In [14]:
stamp = datetime(2011, 1, 3)

In [15]:
str(stamp)

'2011-01-03 00:00:00'

In [16]:
stamp.strftime('%Y-%m-%d') # strftime is string format time? time->str?

'2011-01-03'

In [17]:
value = '2011-01-03'
datetime.strptime(value, '%Y-%m-%d') # strptime is string parse time? str->time?

datetime.datetime(2011, 1, 3, 0, 0)

In [18]:
datestrs = ['7/6/2011','8/6/2011']
[datetime.strptime(x, '%m/%d/%Y') for x in datestrs]

[datetime.datetime(2011, 7, 6, 0, 0), datetime.datetime(2011, 8, 6, 0, 0)]

The strptime method is the 'best' way to parse dates, IF the dates have a known format. The third party dateutil package has a parser.parse method that can be useful if the format isn't (as) known.

The parse method can also sometimes return a date when you might not want it to - for example, it'll turn '42' into the year 2042 w/ today's calendar date.

In [19]:
from dateutil.parser import parse

In [20]:
parse('2011-01-03')

datetime.datetime(2011, 1, 3, 0, 0)

In [21]:
parse('Jan 31, 1997 10:45 PM')

datetime.datetime(1997, 1, 31, 22, 45)

In [23]:
parse('6/12/2011', dayfirst=True)
# dayfirst is needed because parse guesses the format - we don't specify where the month is

datetime.datetime(2011, 12, 6, 0, 0)

In [24]:
datestrs

['7/6/2011', '8/6/2011']

In [25]:
pd.to_datetime(datestrs)

DatetimeIndex(['2011-07-06', '2011-08-06'], dtype='datetime64[ns]', freq=None, tz=None)

In [26]:
idx = pd.to_datetime(datestrs + [None])
idx

DatetimeIndex(['2011-07-06', '2011-08-06', 'NaT'], dtype='datetime64[ns]', freq=None, tz=None)

In [27]:
idx[2]

NaT

In [28]:
pd.isnull(idx)

array([False, False,  True], dtype=bool)